In [13]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from efficientnet.keras import EfficientNetB0
from sklearn.decomposition import PCA
from sklearn.svm import SVC



# Define the path to your dataset
dataset_path = 'C:/Users/mohiu/OneDrive/Python/brain'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

def preprocess_and_augment_data(data, labels):
    # Data augmentation (you can customize this)
    datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load and preprocess the dataset
data, labels = load_dataset(dataset_path)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define a function to build a CNN model
def build_cnn_model(hp):
    model = keras.Sequential()

    # Add EfficientNetB0 base (transfer learning)
    base_model = efn.EfficientNetB0(input_shape=(image_size[0], image_size[1], 3), include_top=False, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())

    # Add dense layers and Using Softmax
    model.add(layers.Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for label encoding
                  metrics=['accuracy'])
    
    return model

# Initialize Bayesian Optimization for hyperparameter tuning
tuner = BayesianOptimization(
    build_cnn_model,
    objective='val_accuracy',
    max_trials=10,
    directory='bayesian_opt',
    project_name='cnn_tuner'
)


# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final CNN model with the best hyperparameters
best_model = build_cnn_model(best_hps)





# Train the model on the full dataset
best_model.fit(data, labels, epochs=10)

# Evaluate the CNN model
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Test accuracy (CNN): {accuracy}")

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=best_model.input, outputs=best_model.layers[-2].output)
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

# Calculate the number of components for PCA
n_components = min(train_features.shape[0], train_features.shape[1])

# Dimensionality reduction using PCA
pca = PCA(n_components=n_components)
train_features_pca = pca.fit_transform(train_features)
test_features_pca = pca.transform(test_features)

# Train an SVM on the extracted features
svm = SVC(C=1.0, kernel='rbf', gamma='auto')
svm.fit(train_features_pca, y_train)

# Evaluate the SVM on the test features
svm_accuracy = svm.score(test_features_pca, y_test)
print(f"SVM Test accuracy: {svm_accuracy}")


# Train a Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_features, y_train)

# Evaluate Random Forest on the test features
rf_accuracy = rf.score(test_features, y_test)
print(f"Random Forest Test accuracy: {rf_accuracy}")

# Train a K Nearest Neighbors (KNN) classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features, y_train)

# Evaluate KNN on the test features
knn_accuracy = knn.score(test_features, y_test)
print(f"KNN Test accuracy: {knn_accuracy}")

# Train an AdaBoost classifier
ada_boost = AdaBoostClassifier(n_estimators=50, random_state=42)
ada_boost.fit(train_features, y_train)

# Evaluate AdaBoost on the test features
ada_boost_accuracy = ada_boost.score(test_features, y_test)
print(f"AdaBoost Test accuracy: {ada_boost_accuracy}")





Reloading Tuner from bayesian_opt\cnn_tuner\tuner0.json
Epoch 1/10
8/8 [==============================] - 6s 242ms/step - loss: 0.6846 - accuracy: 0.6522
Epoch 2/10
8/8 [==============================] - 2s 240ms/step - loss: 0.4360 - accuracy: 0.8221
Epoch 3/10
8/8 [==============================] - 2s 235ms/step - loss: 0.3521 - accuracy: 0.8577
Epoch 4/10
8/8 [==============================] - 2s 236ms/step - loss: 0.3016 - accuracy: 0.8775
Epoch 5/10
8/8 [==============================] - 2s 262ms/step - loss: 0.2906 - accuracy: 0.8933
Epoch 6/10
8/8 [==============================] - 2s 236ms/step - loss: 0.2871 - accuracy: 0.8854
Epoch 7/10
8/8 [==============================] - 2s 247ms/step - loss: 0.2645 - accuracy: 0.8814
Epoch 8/10
8/8 [==============================] - 2s 248ms/step - loss: 0.2532 - accuracy: 0.9012
Epoch 9/10
8/8 [==============================] - 2s 265ms/step - loss: 0.2294 - accuracy: 0.9249
Epoch 10/10
2/2 [==============================] - 1s 168ms/st

In [5]:
!pip install efficientnet


     -------------------------------------- 50.7/50.7 kB 860.9 kB/s eta 0:00:00


In [4]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3, Xception
from efficientnet.tfkeras import EfficientNetB0

# Define the path to the original dataset
original_dataset_path = 'C:/Users/mohiu/OneDrive/Python/brain'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

def preprocess_and_augment_data(data, labels):
    # Data augmentation (you can customize this)
    datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load and preprocess the original dataset
data, labels = load_dataset(original_dataset_path)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Feature Extraction using various models
feature_extractors = [
    VGG19(weights='imagenet', include_top=False, input_shape=(*image_size, 3)),
    EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*image_size, 3)),
    ResNet50(weights='imagenet', include_top=False, input_shape=(*image_size, 3)),
    InceptionV3(weights='imagenet', include_top=False, input_shape=(*image_size, 3)),
    Xception(weights='imagenet', include_top=False, input_shape=(*image_size, 3)),
    build_custom_model((*image_size, 3), num_classes)
]

# Extract features
extracted_features = []
for model in feature_extractors:
    features = model.predict(X_train)
    extracted_features.append(features)

# Two-stage ensemble for selecting best feature extractors (here, simply averaging features)
ensemble_features = np.mean(extracted_features, axis=0)

# You can perform classification on ensemble_features and proceed with further steps

# Save the code to regenerate the data loading and feature extraction
regenerated_code = f'''
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

# Load preprocessed data
loaded_data = np.load('preprocessed_data.npz')
X_train = loaded_data['X_train']
X_test = loaded_data['X_test']
y_train = loaded_data['y_train']
y_test = loaded_data['y_test']

# Define your custom model for scratch-built feature extraction
def build_custom_model(input_shape, num_classes):
    model = keras.Sequential([
        # Define your layers here for the custom model
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model
'''

# Save the code to regenerate the data loading and feature extraction
regenerated_code_path = 'regenerate_code.py'
with open(regenerated_code_path, 'w') as file:
    file.write(regenerated_code)

print("Feature extraction and ensemble completed.")
print(f"Regenerate code saved to: {regenerated_code_path}")


53592064/83683744 [==================>...........] - ETA: 1:05

KeyboardInterrupt: 